In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error,r2_score
import talib
from sklearn.model_selection import train_test_split

In [2]:
data=yf.download('0017.hk','2019-01-01','2022-04-01')
# data.drop('Volume',1,inplace=True)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,41.520000,41.599998,40.080002,40.560001,34.006569,3099411
2019-01-03,40.560001,41.360001,40.240002,40.720001,34.140720,2768268
2019-01-04,40.560001,43.119999,40.320000,42.720001,35.817574,5637624
2019-01-07,42.959999,43.599998,42.880001,43.360001,36.354164,3711498
2019-01-08,43.599998,45.040001,43.439999,44.720001,37.494427,7029280
...,...,...,...,...,...,...
2022-03-25,32.000000,32.900002,31.850000,32.450001,32.450001,4041318
2022-03-28,32.400002,32.400002,31.799999,32.349998,32.349998,1827666
2022-03-29,32.349998,32.400002,31.799999,32.049999,32.049999,2078099


In [3]:
data['Prediction']=data['Close'].shift(-1)
data.dropna(inplace=True)

In [4]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestRegressor
#k_features=10 (It will get top 10 features best suited for prediction)
#forward=False (Backward feature selection model)
#verbose=2 (It will show details output as shown below.)
#cv=5 (Kfold cross valiation: it will split the training set in 5 set and 4 will be using for training the model and 1 will using as validation)
#n_jobs=-1 (Number of cores it will use for execution.-1 means it will use all the cores of CPU for execution.)
#scoring='r2'(R-squared is a statistical measure of how close the data are to the fitted regression line)
backwardModel=sfs(RandomForestRegressor(),k_features=10,forward=True,verbose=2,cv=5,n_jobs=-1,scoring='r2')
#We will convert our training data into numpy array. If we will not convert it, model is not able to read some of the column names.
backwardModel.fit(np.array(data.drop('Prediction',1)),data['Prediction'].values.ravel())

/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_47382/3806124569.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  backwardModel.fit(np.array(data.drop('Prediction',1)),data['Prediction'].values.ravel())


AttributeError: k_features must be a positive integer between 1 and X.shape[1], got 10

In [ ]:
# data.drop(['OSC12','OSC6','MI12','MI6','%R10','%R5','RSI','TR'],1,inplace=True)
data_val=data[-100:]
data=data[:-100]
sc = MinMaxScaler(feature_range = (-1, 1))
# X=sc.fit_transform(data.drop('Prediction',1))
X=data.drop('Prediction',1).values
# Y = data["Prediction"].values.reshape(-1,1)
# Y = sc_y.fit_transform(data["Prediction"].values.reshape(-1,1))
Y=np.array(data["Prediction"])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
# X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
# X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
clf = RandomForestRegressor()

In [ ]:
model = clf.fit(X_train, y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
mean_absolute_percentage_error(y_test,y_pred)

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
# y_test = sc.inverse_transform(y_test)
# y_pred = sc.inverse_transform(y_pred)
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.title('HSI-RF-Scaling')
plt.legend()
plt.show();

In [ ]:
mean_absolute_percentage_error(y_test,y_pred)

In [ ]:
mean_squared_error(y_test,y_pred,squared=False)

In [ ]:
r2_score(y_test,y_pred)